# Cluster Galaxy Classifier - Student Notebook

Use this notebook to identify cluster member galaxies in your own cluster images using a pre-trained model.

## Instructions:
1. Run each cell in order (click the play button or press Shift+Enter)
2. Upload your cluster image when prompted
3. View and download your results!

## Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install -q numpy scipy pandas scikit-learn scikit-image opencv-python matplotlib astropy joblib

## Step 2: Download the Code and Model

In [ ]:
# Clone the repository
!git clone https://github.com/ASPIRONOMY/cluster-galaxy-classifier.git
%cd cluster-galaxy-classifier

## Step 3: Upload Your Cluster Image

In [ ]:
from google.colab import files
import os

# Upload your cluster image
print("Upload your cluster image (PNG or JPG):")
uploaded = files.upload()

# Get the uploaded file name
image_filename = list(uploaded.keys())[0]
print(f"\n[OK] Uploaded: {image_filename}")

## Step 4: Classify Galaxies in Your Image

In [ ]:
# Import the pipeline
from complete_pipeline import run_complete_pipeline
from pathlib import Path

# Create results directory
Path('results').mkdir(exist_ok=True)

# Run classification on your image
print("="*60)
print("CLASSIFYING GALAXIES IN YOUR CLUSTER")
print("="*60)

results = run_complete_pipeline(
    image_file=image_filename,
    model_dir='models',  # Uses pre-trained model
    output_dir='results',
    detection_method='comprehensive',  # Best for detecting all objects including large ones
    detection_params={
        'threshold_percentile': 75,  # Lower threshold to catch brighter objects
        'min_area': 5,
        'max_area': None  # No upper limit - detects all sizes
    },
    confidence_threshold=0.6  # Start with 0.6 (try 0.5-0.8 to adjust sensitivity)
)

print(f"\n[OK] Classification complete!")
print(f"Detected {len(results['detected_galaxies'])} total galaxies")
print(f"Classified {len(results['cluster_members'])} as cluster members")
print(f"Classified {len(results['non_members'])} as non-members")

## Step 5: View Results

In [ ]:
from IPython.display import Image, display
import pandas as pd

# Display the visualization
image_name = Path(image_filename).stem
viz_file = f'results/{image_name}_classified.png'
csv_file = f'results/{image_name}_members.csv'

if Path(viz_file).exists():
    print("Visualization (Red circles = Cluster Members, Blue circles = Non-Members):")
    display(Image(viz_file))
else:
    print(f"Visualization file not found: {viz_file}")

In [ ]:
# Show coordinates of cluster members
if Path(csv_file).exists():
    members_df = pd.read_csv(csv_file)
    print(f"\nCluster Member Coordinates ({len(members_df)} members):")
    display(members_df.head(20))  # Show first 20
    if len(members_df) > 20:
        print(f"\n... and {len(members_df) - 20} more members")
    
    # Show probability statistics for debugging
    if 'probability' in members_df.columns:
        print(f"\nProbability statistics for members:")
        print(f"  Min: {members_df['probability'].min():.3f}")
        print(f"  Max: {members_df['probability'].max():.3f}")
        print(f"  Mean: {members_df['probability'].mean():.3f}")
else:
    print(f"CSV file not found: {csv_file}")

## Step 6: Download Your Results

## Debug: Check Probability Distribution (Optional)

If you're getting unexpected results, run this cell to see the probability distribution:

In [ ]:
# Debug: Check probability distribution of all detected objects
import pandas as pd
import numpy as np

# Try to find the full results file (may have different naming)
results_dir = Path('results')
csv_files = list(results_dir.glob('*_members.csv'))

if csv_files:
    # Load the most recent CSV
    debug_df = pd.read_csv(csv_files[0])
    
    print("="*60)
    print("PROBABILITY DISTRIBUTION ANALYSIS")
    print("="*60)
    print(f"\nTotal objects detected: {len(debug_df)}")
    
    if 'probability' in debug_df.columns:
        print(f"\nProbability Statistics:")
        print(debug_df['probability'].describe())
        
        print(f"\nObjects by probability threshold:")
        print(f"  >= 0.5: {len(debug_df[debug_df['probability'] >= 0.5])}")
        print(f"  >= 0.6: {len(debug_df[debug_df['probability'] >= 0.6])}")
        print(f"  >= 0.7: {len(debug_df[debug_df['probability'] >= 0.7])}")
        print(f"  >= 0.8: {len(debug_df[debug_df['probability'] >= 0.8])}")
        
        print(f"\nRecommendation:")
        if len(debug_df[debug_df['probability'] >= 0.6]) < 10:
            print("  ⚠️  Very few objects above 0.6 threshold.")
            print("  💡 Try lowering confidence_threshold to 0.5 or 0.55")
        elif len(debug_df[debug_df['probability'] >= 0.7]) > len(debug_df[debug_df['probability'] >= 0.6]) * 0.8:
            print("  ✅ Most objects have high confidence. Current threshold (0.6) is good.")
        else:
            print("  💡 Consider adjusting confidence_threshold based on your needs")
    else:
        print("No 'probability' column found in results")
else:
    print("No results CSV files found")

In [ ]:
# Download the visualization image
if Path(viz_file).exists():
    print("Downloading visualization image...")
    files.download(viz_file)
    print("[OK] Visualization downloaded!")
else:
    print("Visualization file not found")

# Download the CSV file with coordinates
if Path(csv_file).exists():
    print("\nDownloading coordinates CSV...")
    files.download(csv_file)
    print("[OK] Coordinates downloaded!")
else:
    print("CSV file not found")

## Tips

- **Confidence threshold**: Adjust `confidence_threshold` (0.5-0.9) to control sensitivity
  - `0.6` = More sensitive (default in this notebook)
  - `0.7` = Balanced
  - `0.8` = Very conservative (fewer false positives)
  - `0.5` = Most sensitive (may include false positives)
- **Detection method**: `'comprehensive'` is recommended (detects all sizes including large objects)
- **Detection parameters**: The notebook uses explicit parameters to ensure consistent results
- **Image format**: PNG or JPG RGB images work best

## Troubleshooting

**"Model file not found"**:
- Make sure the repository was cloned correctly
- Check that the `models/` folder exists with model files

**"No galaxies detected"**:
- Try adjusting detection parameters
- Make sure your image is clear and not corrupted

**"Too few members detected"**:
- Lower `confidence_threshold` to 0.5 or 0.55
- Check the probability statistics in the results - if most probabilities are below 0.6, the model may need retraining

**"Too many false positives"**:
- Increase `confidence_threshold` to 0.7, 0.8, or 0.9
- Check the probability statistics - if many have low probabilities (<0.7), increase the threshold